In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.base import Int, Str, Float, Bool
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.scanning_probe.afm.afm_workchain import AfmWorkChain

# Select structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    update_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
clear_output()

display(ipw.VBox([struct_browser, viewer]))

def update_view():
    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
        
    structure = struct_browser.results.value
    if not structure:
        return
    
    atoms = structure.get_ase()
    atoms.pbc = [1, 1, 1]
    #print("CELL:", atoms.cell)
    # add new component
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()
    

# Select computer and codes

In [ ]:
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True})
computer_names = [comp[0].name for comp in qb.all()]

style = {'description_width': '140px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': plugin_name})
    qb.order_by({Code: {'id': 'desc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    afm_codes = comp_plugin_codes(drop_computer.value, 'spm.afm')
    
    # Clear as there seems to be a bug when label remains same,
    # then so does the value
    drop_cp2k.options = []
    drop_pp.options = []
    drop_2pp.options = []
    
    drop_cp2k.options = [(c.label, c) for c in cp2k_codes]
    drop_pp.options = [(c.label, c) for c in afm_codes if "_pp" in c.label]
    drop_2pp.options = [(c.label, c) for c in afm_codes if "_2pp" in c.label]
    
    
drop_computer.observe(on_computer_change, names='value')

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_pp = ipw.Dropdown(description="AFM PP")

drop_2pp = ipw.Dropdown(description="AFM 2PP")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(drop_computer, drop_cp2k, elpa_check, drop_pp, drop_2pp)

# AFM parameters

In [ ]:
layout_small = {'width': '30%'}

scanstep_floattxt = ipw.BoundedFloatText(
                        description='Scan dx (ang)',
                        min=0.05,
                        max=0.5,
                        step=0.05,
                        value=0.1,
                        style=style, layout=layout_small)

scanminz_floattxt = ipw.BoundedFloatText(
                        description='Scan min z (ang)',
                        min=0.0,
                        max=5.0,
                        step=0.1,
                        value=3.5,
                        style=style, layout=layout_small)

scanmaxz_floattxt = ipw.BoundedFloatText(
                        description='Scan max z (ang)',
                        min=5.0,
                        max=10.0,
                        step=0.1,
                        value=8.5,
                        style=style, layout=layout_small)

amp_floattxt = ipw.FloatText(
                        description='Amplitude (ang)',
                        step=0.1,
                        value=1.4,
                        style=style, layout=layout_small)

f0_cantilever_floattxt = ipw.FloatText(
                        description='Cantilever f0',
                        step=0.1,
                        value=22352.5,
                        style=style, layout=layout_small)


display(scanstep_floattxt, scanminz_floattxt, scanmaxz_floattxt, amp_floattxt, f0_cantilever_floattxt)

In [ ]:
drop_2pp_resp = ipw.Dropdown(description="2PP RESP model",
                            style=style, layout=layout_small,
                            options = { # ChargeCuUp, ChargeCuDown, Ccharge, Ocharge
                                'pentacene': [-0.0669933, -0.0627402, 0.212718, -0.11767],
                                'ptcda':     [     -0.05,      -0.07,     0.23,    -0.13]
                            })

display(drop_2pp_resp)

In [ ]:
def create_pp_parameterdata(ase_geom):
    cell = ase_geom.cell
    top_z = np.max(ase_geom.positions[:, 2])
    dx = scanstep_floattxt.value
    paramdata = ParameterData(dict={
        'probeType':    'Otip',
        'charge':       -0.028108681223969645,
        'sigma':        0.7,
        'tip':          's',
        'klat':         0.34901278868090491,
        'krad':         21.913190531846034,
        'r0Probe':      [0.0, 0.0, 2.97],
        'PBC':          'False',
        'gridA':        list(cell[0]),
        'gridB':        list(cell[1]),
        'gridC':        list(cell[2]),
        'scanMin':      [0.0, 0.0, np.round(top_z, 1)+scanminz_floattxt.value],
        'scanMax':      [cell[0,0], cell[1,1], np.round(top_z, 1)+scanmaxz_floattxt.value],
        'scanStep':     [dx, dx, dx],
        'Amplitude':    amp_floattxt.value,
        'f0Cantilever': f0_cantilever_floattxt.value
    })
    return paramdata

def create_2pp_parameterdata(ase_geom):
    cell = ase_geom.cell
    top_z = np.max(ase_geom.positions[:, 2])
    dx = scanstep_floattxt.value
    resp = drop_2pp_resp.value
    paramdata = ParameterData(dict={
        'Catom':        'Ctip',
        'Oatom':        'Otip',
        'ChargeCuUp':   resp[0],
        'ChargeCuDown': resp[1],
        'Ccharge':      resp[2],
        'Ocharge':      resp[3],
        'sigma':        0.7,
        'Cklat':        0.24600212465950813,
        'Oklat':        0.15085476515590224,
        'Ckrad':        20,
        'Okrad':        20,
        'rC0':          [0.0, 0.0, 1.82806112489999961213],
        'rO0':          [0.0, 0.0, 1.14881347770000097341],
        'PBC':          'False',
        'gridA':        list(cell[0]),
        'gridB':        list(cell[1]),
        'gridC':        list(cell[2]),
        'scanMin':      [0.0, 0.0, np.round(top_z, 1)+scanminz_floattxt.value],
        'scanMax':      [cell[0,0], cell[1,1], np.round(top_z, 1)+scanmaxz_floattxt.value],
        'scanStep':     [dx, dx, dx],
        'Amplitude':    amp_floattxt.value,
        'f0Cantilever': f0_cantilever_floattxt.value
    })
    return paramdata

In [ ]:
def find_struct_wf(structure):
    selected_comp = qb.all()[drop_computer.index][0]
    
    # check stm
    extras = structure.get_extras()
    for ex_k in extras.keys():
        if ex_k.startswith('stm'):
            stm_workchain = load_node(extras[ex_k])
            cp2k_scf_calc = stm_workchain.get_outputs()[0]
            if cp2k_scf_calc.get_computer().hostname == selected_comp.hostname:
                wfn_path = cp2k_scf_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
                # check if it exists
                file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
                if file_exists[0]:
                    print("Found .wfn from %s"%ex_k)
                    return wfn_path
    
    # check geo opt
    geo_opt_calc = structure.get_inputs()[0]
    geo_comp = geo_opt_calc.get_computer()
    if geo_comp.hostname == selected_comp.hostname:
        wfn_path = geo_opt_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
        # check if it exists
        file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
        if file_exists[0]:
            print("Found .wfn from geo_opt")
            return wfn_path
    return ""

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        if not drop_cp2k.value or not drop_pp.value or not drop_2pp.value:
            print("Please select all the codes.")
            return
        
        struct = struct_browser.results.value
        
        ase_geom = struct.get_ase()
        cell_array = ArrayData()
        cell_array.set_array('cell', np.diag(ase_geom.cell))

        afm_pp_params = create_pp_parameterdata(ase_geom)
        afm_2pp_params = create_2pp_parameterdata(ase_geom)
        
        ## Try to access the restart-wfn file ##
        try:
            wfn_file_path = find_struct_wf(struct)
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Didn't find any accessible .wfn file.")
        
        outputs = submit(AfmWorkChain,
                 cp2k_code=drop_cp2k.value,
                 structure=struct,
                 cell=cell_array,
                 wfn_file_path=Str(wfn_file_path),
                 elpa_switch=Bool(elpa_check.value),
                 afm_pp_code=drop_pp.value,
                 afm_pp_params=afm_pp_params,
                 afm_2pp_code=drop_2pp.value,
                 afm_2pp_params=afm_2pp_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)